In [1]:
import pandas as pd
from sklearn import model_selection

from sklearn.metrics import accuracy_score, precision_score, recall_score, log_loss, accuracy_score, make_scorer

from sklearn.model_selection import train_test_split, GridSearchCV

from keras.models import Sequential, Input
from keras.layers import  Dropout, Dense, Activation
from keras.wrappers.scikit_learn import KerasClassifier

from tensorflow.keras import layers

from sklearn.linear_model import Perceptron
from sklearn.neural_network import MLPRegressor, MLPClassifier

In [2]:
df_train_mean_encoding = pd.read_csv('../../Feature_Encoding/data/train_mean_encoding.csv')
df_test_mean_encoding = pd.read_csv('../../Feature_Encoding/data/test_mean_encoding.csv')
train = pd.read_csv('../../Feature_Engineering/data/other-cleaned_train.csv')

In [3]:
def cross_val(model, x_train, y_train):
    score_cross_val = model_selection.cross_val_score(model, x_train, y_train, cv=5)
    print(score_cross_val.mean())

In [4]:
# Red neuronal profunda - https://keras.io/guides/sequential_model/
# nFeatures -> cantidad de features (columnas de set)
# nClasses -> cantidad de clases que pueden ser (2 -> lost vs won)
def DNN_model_bin(optimizer='rmsprop',init='glorot_uniform'):
    node = 512
    nClasses = 2
    dropout=0.5
    nFeatures = 185
    
    model = Sequential()
    model.add(Dense(node,input_dim=nFeatures,activation='relu'))
    model.add(Dropout(dropout))
    for i in range(0,4):
        model.add(Dense(node,input_dim=node,activation='relu'))
        model.add(Dropout(dropout))
    model.add(Dense(nClasses, activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model
    
# Red neuronal profunda - https://keras.io/guides/sequential_model/
# nFeatures -> cantidad de features (columnas de set)
# nClasses -> cantidad de clases que pueden ser (2 -> lost vs won)
def DNN_model(optimizer='rmsprop',init='glorot_uniform'):
    node = 512
    nClasses = 2
    dropout=0.5
    nFeatures = 55
    
    model = Sequential()
    model.add(Dense(node,input_dim=nFeatures,activation='relu'))
    model.add(Dropout(dropout))
    for i in range(0,4):
        model.add(Dense(node,input_dim=node,activation='relu'))
        model.add(Dropout(dropout))
    model.add(Dense(nClasses, activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model
 

In [5]:
def dnn(x_train, y_train, x_validation, y_validation):
    keras_model = KerasClassifier(build_fn=DNN_model)
    optimizers = ['rmsprop', 'adam']
    init = ['glorot_uniform', 'normal', 'uniform']
    epochs = [50, 100, 150]
    batches = batches = [5, 10, 20]
    param_grid = dict(optimizer=optimizers, nb_epoch=epochs, batch_size=batches, init=init)
    dnn_gs = GridSearchCV(keras_model, param_grid=param_grid)
    dnn_gs.fit(x_train, y_train)
    dnn_best = dnn_gs.best_estimator_
    print(dnn_gs.best_params_)
    print('dnn: {}'.format(dnn_best.score(x_validation, y_validation)))
    return dnn_best

def dnn_bin(x_train, y_train, x_validation, y_validation):
    keras_model = KerasClassifier(build_fn=DNN_model_bin)
    optimizers = ['rmsprop', 'adam']
    init = ['glorot_uniform', 'normal', 'uniform']
    epochs = [50, 100, 150]
    batches = batches = [5, 10, 20]
    param_grid = dict(optimizer=optimizers, nb_epoch=epochs, batch_size=batches, init=init)
    dnn_gs = GridSearchCV(keras_model, param_grid=param_grid)
    dnn_gs.fit(x_train, y_train)
    dnn_best = dnn_gs.best_estimator_
    print(dnn_gs.best_params_)
    print('dnn: {}'.format(dnn_best.score(x_validation, y_validation)))
    return dnn_best

In [6]:
def test_model(model, x_test, y_test):
    predictions = model.predict_proba(x_test)[:,1]
    logloss = log_loss(y_test, predictions)
    accuracy = accuracy_score(y_test, predictions.round())
    print("Accuracy: %.2f%%, Logloss: %.2f" % (accuracy*100.0, logloss))

In [7]:
y = train.Target
x_train_mean_encoding, x_validation_mean_encoding, y_train_mean_encoding, y_validation_mean_encoding = train_test_split(df_train_mean_encoding, y, test_size=0.3, stratify=y)

## Mean Encoding

In [8]:
DNN_mean = dnn(x_train_mean_encoding,y_train_mean_encoding,x_validation_mean_encoding,y_validation_mean_encoding)

473/473 [==============================] - 3s 6ms/step - loss: 1091.3331 - accuracy: 0.5588


473/473 [==============================] - 3s 6ms/step - loss: 1612.8801 - accuracy: 0.6654


946/946 [==============================] - 28s 29ms/step - loss: 74956.0083 - accuracy: 0.4994


237/237 [==============================] - 2s 9ms/step - loss: 110.3611 - accuracy: 0.6578


473/473 [==============================] - 17s 34ms/step - loss: 45969.5567 - accuracy: 0.5033


119/119 [==============================] - 1s 8ms/step - loss: 4791.2969 - accuracy: 0.5527


2364/2364 [==============================] - 71s 29ms/step - loss: 58211.1732 - accuracy: 0.5160
{'batch_size': 5, 'init': 'normal', 'nb_epoch': 50, 'optimizer': 'adam'}
1013/1013 [==============================] - 6s 6ms/step - loss: 1311.8948 - accuracy: 0.5828
dnn: 0.5828232765197754


In [9]:
test_model(DNN_mean,x_validation_mean_encoding,y_validation_mean_encoding)
cross_val(DNN_mean, x_train_mean_encoding, y_train_mean_encoding)

/home/violeta/.local/lib/python3.8/site-packages/tensorflow/python/keras/engine/sequential.py:425: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '
/home/violeta/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:2279: RuntimeWarning: divide by zero encountered in log
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)
/home/violeta/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:2279: RuntimeWarning: invalid value encountered in multiply
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)


Accuracy: 58.28%, Logloss: nan
473/473 [==============================] - 3s 6ms/step - loss: 3445.0415 - accuracy: 0.5696
0.5921504855155945


## PERCEPTRON

In [10]:
def perceptron(x_train, y_train, x_validation, y_validation):
    perceptron = Perceptron(tol=1e-3, random_state=0)
    params_perc = {'alpha': [0.0001, 0.0003, 0.001, 0.003, 0.01, 0.03, 0.1, 0.3]}
    per_gs = model_selection.GridSearchCV(perceptron, params_perc, cv=5)
    per_gs.fit(x_train, y_train)
    per_best = per_gs.best_estimator_
    print(per_gs.best_params_)
    print('perceptron: {}'.format(per_best.score(x_validation, y_validation)))
    return per_gs

def multi_perceptron(x_train, y_train, x_validation, y_validation):
    mult_perceptron = MLPClassifier(tol=1e-3, random_state=0)
    params_mult_perc =  {'hidden_layer_sizes': [(10,30,10),(20,)],'activation': ['tanh', 'relu'],'solver': ['sgd', 'adam'],
    'alpha': [0.0001, 0.05],'learning_rate': ['constant','adaptive'],
    }
    mul_per_gs = model_selection.GridSearchCV(mult_perceptron, params_mult_perc, cv=5)
    mul_per_gs.fit(x_train, y_train)
    mul_per_best = mul_per_gs.best_estimator_
    print(mul_per_gs.best_params_)
    print('multi perceptron: {}'.format(mul_per_best.score(x_validation, y_validation)))
    return mul_per_best

def multi_perceptron_2(x_train, y_train, x_validation, y_validation):
    mult_perceptron_2 = MLPRegressor()
    params_mult_perc_2 = {'hidden_layer_sizes': [(50,50,50), (50,100,50), (20,)],'activation': ['relu','tanh','logistic'],
          'alpha': [0.0001, 0.05],'learning_rate': ['constant','adaptive'],
          'solver': ['sgd', 'adam']}
    mul_per_gs_2 = model_selection.GridSearchCV(mult_perceptron_2, params_mult_perc_2, cv=5)
    mul_per_gs_2.fit(x_train, y_train)
    mul_per_best_2 = mul_per_gs_2.best_estimator_
    print(mul_per_gs_2.best_params_)
    print('multi perceptron 2: {}'.format(mul_per_best_2.score(x_validation, y_validation)))
    return mul_per_gs_2

In [11]:
PERC_mean = multi_perceptron(x_train_mean_encoding,y_train_mean_encoding,x_validation_mean_encoding,y_validation_mean_encoding)
test_model(PERC_mean,x_validation_mean_encoding,y_validation_mean_encoding)
cross_val(PERC_mean, x_train_mean_encoding, y_train_mean_encoding)

{'activation': 'tanh', 'alpha': 0.0001, 'hidden_layer_sizes': (20,), 'learning_rate': 'constant', 'solver': 'adam'}
multi perceptron: 0.5646594274432379
Accuracy: 56.47%, Logloss: 0.67
0.6711795568024529
